# 1. Boîte à outils
On va d’abord constituer une petite bibliothèque de calcul modulo un nombre premier, permettant de réaliser :
* L’inversion modulo `p` (via l’algorithme d’Euclide étendu)
* Un crible pour construire la liste des nombres premiers inférieurs à une certaine borne.
L’algorithme classique est le suivant : partir d’un tableau de taille `n`, et marquer les cases multiples de `i` où `i > 1` est le numéro de la plus petite case non marquée. À la fin, les cases non marquées sont aux positions premières. Vous pourrez générer la liste des premiers jusqu’à `1000000` et la stocker dans un fichier.
* Un test de primalité.
* Le calcul des diviseurs premiers d’un entier donné.
* Le calcul d’un générateur de $\mathbb{Z}/p\mathbb{Z}^*$ (on pourra tirer un élément au hasard tant qu’on ne trouve pas de générateur).
* L’exponentiation modulo un nombre premier.

In [ ]:
import random
import math
from math import sqrt


def euclid(a, b):
    if b == 0:
        return a, 1, 0
    (pgcd1, u1, v1) = euclid(b, a % b)  # Récursivité
    pgcd, u, v = pgcd1, v1, u1 - v1 * (a // b)
    return (pgcd, u, v)  # PGCD, coeff de bézout


def inv(a, n):
    # renvoie l'inverse de a modulo n, ou une erreur si a n'est pas inversible
    pgcd, x, y = euclid(a, n)
    if pgcd != 1:
        # a is not invertible modulo n
        raise ValueError(f"{a} is not invertible modulo {n}")
    else:
        return x % n


def sieve(n):
    # calcule la liste des nombres premiers entre 1 et n
    prime = [True for i in range(n + 1)]
    prime[0] = False
    prime[1] = False

    for i in range(2, n + 1):
        if prime[i]:
            for j in range(i*2, n + 1, i):
                prime[j] = False

    return [i for i in range(n + 1) if prime[i]]


def is_prime(n):
    # teste si n est premier
    for i in range(2, int(sqrt(n)) + 1):
        if (n % i) == 0:
            return False
    return True


def div(n):
    # renvoie la liste des diviseurs premiers de n
    L=[]
    for i in range(2,int(sqrt(n))+3):
        if n%i==0:
            if is_prime(i)==True:
                L.append(i)
    return L


def exp(a, b, n):
    # calcule `a` ^ `b` mod `n` (en log(n) multiplications)
    # initialiser le résultat à 1
    res = 1

    # utiliser l'exponentiation rapide pour calculer a^b mod n
    while b > 0:
        if b % 2 == 1:
            res = (res * a) % n
        a = (a * a) % n
        b = b // 2

    return res


def gen(n):
    # calcule un générateur de (Z/nZ)*
    # On tire un entier au hasard jusqu'à ce que ce soit un générateur
    # Pour tester si g est générateur, on calcule g^((n-1)/d) mod n pour chaque diviseur premier d de n-1.
    # Si on tombe sur 1, g n'est pas générateur
    cond = True
    divs = div(n - 1)
    while cond:
        g = random.randint(1, n - 1)
        cond = any([exp(g, (n - 1) / d, n) == 1 for d in divs])
    return g




# 2. Diffie-Hellman

Implémenter une communication entre Arielle et Bertrand sous TCP (on
pourra considérer que l'un des deux est un serveur et l'autre un
client) permettant d'appliquer le protocole de Diffie-Hellman. Faites
afficher à chaque participant les étapes du protocole sur sa sortie
standard.

In [ ]:
import threading
import socket as sckt

n = 100000
primes = sieve(n)
p = primes[len(primes) - 1]
print("notre premier : %d", p)
g = gen(n)
print("notre générateur : %d", g)

def bertrand(port):
    def bertrand_behavior(port):
        # Bertrand choisit `b` sa clé privée
        b = random.randint(1,n-1)

        socket = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
        socket.bind(("127.0.0.1", port))

        socket.listen(True)

        print("Bertrand : initialized on port {}".format(port))
        print("Bertrand : Listening")
        conn, addr = socket.accept()

        print('Bertrand : Got new connection from {} at {}'.format(addr[0], addr[1]))

        # Bertrand attend qu'Arielle lui envoie g^a modulo p
        ga = int(conn.recv(1024).decode())
        print("Bertrand : Received g^a % p : {} from {}".format(ga, addr[1]))

        # Bertrand envoie g^b modulo p à Arielle
        gb = exp(g,b,p)
        print("Bertrand : Sending g^b % p : {} to {}".format(gb, addr[1]))
        conn.send(str(gb).encode())

        # Bertrand détermine la clé commune : g^ab modulo p
        gab = exp(ga, b, p)
        print("Bertrand : calculated key : {}".format(gab))
        conn.close()
        print("Bertrand : Closing the connection from {} at {}".format(addr[0], addr[1]))

        socket.listen(False)
        socket.close()
        print("Bertrand : Disconnected")

    x = threading.Thread(target=bertrand_behavior, args=([port]))
    x.start()


def arielle(port):
    # Arielle choisit `a` sa clé privée
    a = random.randint(1,n-1)

    socket = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
    try:
        socket.connect(("127.0.0.1", port))
        print("Arielle {} : connected".format(socket.getsockname()[1]))
    except:
        print("Arielle : Connection Error")
        raise Exception

    # Arielle envoie g^a modulo p à Bertrand
    ga = exp(g,a,p)
    print("Arielle : Sending g^a : {} to {}".format(ga, port))
    socket.send(str(ga).encode())

    # Arielle attend que Bertrand lui envoie g^b modulo p
    gb = int(socket.recv(1024).decode())
    print("Arielle : Received g^b : {} from {}".format(gb, port))

    # Arielle détermine la clé commune : g^ab modulo p
    gab = exp(gb, a, p)
    print("Arielle : calculated key : {}".format(gab))

    # Déconnexion du client
    socket.close()
    print("Arielle : Disconnected")


port = 1024
bertrand(port)
arielle(port)


Bertrand : running on port 1024
Bertrand : Waiting for a new connection
Bertrand : New connection 127.0.0.1:60276
Arielle : Sending g^a : 0
Bertrand : Received g^a % p : 0
Bertrand : Sending g^b % p : 0
Bertrand : Generated key : 0Arielle : Received g^b : 0
Arielle : Generated key : 0
Arielle : Disconnected

Bertrand : Connection 127.0.0.1:60276 closed
Bertrand : Disconnected


# 3. L’homme du milieu

On se place maintenant dans le cadre d’une attaque par l’homme du milieu.

Pour ne pas rentrerdans les détails techniques de comment l’attaquant peut intercepter les communications, on va se placer dans le cadre où les clients sur les machines d’*Arielle* et *Bertrand* communiquent avec un serveur sur la machine de l’attaquant *Laurent*.*Arielle* et *Bertrand* veulent communiquer en utilisant *RSA*. 

Implémenter le serveur de *Laurent* pour qu’il se fasse passer pour *Bertrand* auprès d’*Arielle* (et inversement). *Laurent* affichera sur lasortie standard les messages échangés par *Arielle* et *Bertrand* (sous forme déchiffrée).

In [ ]:
def arielle():
    # connexion à Laurent
    
    # génération des clés + échange des clés publiques
    
    # communication 
    
def bertrand():
    # connexion à Laurent
    
    # génération des clés + échange des clés publiques
    
    # communication 
    
def laurent():
    # connexion à Arielle et Bertrand
    
    # génération des clés + échange des clés publiques
   
    # affichage des messages interceptés

In [ ]:
import threading
import socket as sckt
import time


def rsa_encrypt(plain_message, pq, e):
    encrypted_message = ""
    for lettre in plain_message:
        asc = ord(lettre)
        asc = exp(asc, e, pq)
        encrypted_message += chr(asc)

    return encrypted_message


def rsa_decrypt(encrypted_message, pq, d):
    plain_message = ""
    for lettre in encrypted_message:
        plain_message += chr(exp(ord(lettre), d, pq))
    return plain_message


def laurent(port):
    # Laurent construit ses clés
    n = 100
    primes = sieve(n)
    primes = primes[len(primes) // 2:]

    p, q = 0, 0
    while (p == q):
        p, q = primes[random.randint(1, len(primes))], primes[random.randint(1, len(primes))]

    pq = p * q
    phi = (p - 1) * (q - 1)
    e = 0
    ok = False
    while (ok == False):
        e = random.randint(1, phi - 1)
        pgcd, x, y = euclid(e, phi)
        if pgcd == 1:
            ok = True

    d = inv(e, phi)

    socket = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
    socket.bind(("127.0.0.1", port))

    socket.listen(True)

    print("Laurent : initialized on port {}".format(port))
    print("Laurent : Listening")

    # première victime
    conn1, addr1 = socket.accept()
    print('Laurent : Got new connection from {} at {}'.format(addr1[0], addr1[1]))
    # réception des clés publiques de la première victime
    pq1 = int(conn1.recv(1024).decode())
    e1 = int(conn1.recv(1024).decode())
    print("Laurent : Received from {} : (pq={}, e={})".format(addr1[1], pq1, e1))

    # envoi des clés publique de Laurent à la première victime
    conn1.send(str(pq).encode())
    time.sleep(1)
    conn1.send(str(e).encode())

    # deuxième victime
    conn2, addr2 = socket.accept()
    print('Laurent : Got new connection from {} at {}'.format(addr2[0], addr2[1]))
    # réception des clés publiques de la deuxième victime
    pq2 = int(conn2.recv(1024).decode())
    e2 = int(conn2.recv(1024).decode())
    print("Laurent : Received from {} : (pq={}, e={})".format(addr2[1], pq2, e2))

    # envoi des clés publique de Laurent à la deuxième victime
    conn2.send(str(pq).encode())
    time.sleep(1)
    conn2.send(str(e).encode())

    def communication(conn_src, addr_src, conn_dest, addr_dest, pq_dest, e_dest):
        # Interception d'un message des victimes
        encrypted_message = conn_src.recv(1024).decode()
        plain_message = rsa_decrypt(encrypted_message, pq, d)
        print("Laurent : Intercepted {} from {}".format(plain_message, addr_src[1]))

        # Transmettre le message à la deuxième victime
        print("Laurent : Forwarding to : {}".format(addr_dest[1]))
        encrypted_message = rsa_encrypt(plain_message, pq_dest, e_dest)
        conn_dest.send(encrypted_message.encode())

    communication1 = threading.Thread(target=communication, args=(conn1, addr1, conn2, addr2, pq2, e2))
    communication1.start()
    communication2 = threading.Thread(target=communication, args=(conn2, addr2, conn1, addr1, pq1, e1))
    communication2.start()
    communication1.join()
    communication2.join()
    conn1.close()
    conn2.close()
    socket.listen(False)
    socket.close()
    print("Laurent : Disconnected")


def bertrand(port):
    def bertrand_behavior(port):
        socket = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
        try:
            socket.connect(("127.0.0.1", port))
            print("Bertrand {}: connected".format(socket.getsockname()[1]))
        except:
            print("Bertrand : Connection Error")

        # Bertrand construit ses clés
        n = 100
        primes = sieve(n)
        primes = primes[len(primes) // 2:]

        p, q = 0, 0
        while (p == q):
            p, q = primes[random.randint(1, len(primes))], primes[random.randint(1, len(primes))]

        pq = p * q
        phi = (p - 1) * (q - 1)
        e = 0
        ok = False
        while (ok == False):
            e = random.randint(1, phi - 1)
            pgcd, x, y = euclid(e, phi)
            if pgcd == 1:
                ok = True

        d = inv(e, phi)

        # Communication des clés public au destinataire
        socket.send(str(pq).encode())
        time.sleep(1)
        socket.send(str(e).encode())

        # Réception des clés publiques du destinataire
        pq_dest = int(socket.recv(1024).decode())
        e_dest = int(socket.recv(1024).decode())
        print("Bertrand : Received from \"Arielle\" : (pq={}, e={})".format(pq_dest, e_dest))

        # Bertrand reçoit le message d'Arielle
        encrypted_message = socket.recv(1024).decode()
        plain_message = rsa_decrypt(encrypted_message, pq, d)
        print("Arielle : Received from \"Bertrand\" : {}".format(plain_message))

        # Bertrand envoie une réponse à Arielle
        plain_message = "Hi"
        encrypted_message = rsa_encrypt(plain_message, pq_dest, e_dest)
        print("Bertrand : Sending to \"Arielle\" : {}".format(encrypted_message))
        socket.send(encrypted_message.encode())

        # Déconnexion du client
        socket.close()
        print("Bertrand : Disconnected")

    x = threading.Thread(target=bertrand_behavior, args=([port]))
    x.start()

    return x


def arielle(port):
    socket = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
    try:
        socket.connect(("127.0.0.1", port))
        print("Arielle {}: connected".format(socket.getsockname()[1]))
    except:
        print("Arielle : Connection Error")

    # Arielle construit ses clés
    n = 100
    primes = sieve(n)
    primes = primes[len(primes) // 2:]

    p, q = 0, 0
    while (p == q):
        p, q = primes[random.randint(1, len(primes))], primes[random.randint(1, len(primes))]

    pq = p * q
    phi = (p - 1) * (q - 1)
    e = 0
    ok = False
    while (ok == False):
        e = random.randint(1, phi - 1)
        pgcd, x, y = euclid(e, phi)
        if pgcd == 1:
            ok = True

    d = inv(e, phi)

    # Communication des clés public au destinataire
    time.sleep(1)
    socket.send(str(pq).encode())
    time.sleep(1)
    socket.send(str(e).encode())

    # Réception des clés publiques du destinataire
    pq_dest = int(socket.recv(1024).decode())
    e_dest = int(socket.recv(1024).decode())
    print("Arielle : Received from \"Bertrand\" : (pq={}, e={})".format(pq_dest, e_dest))

    # Arielle envoie un message à Bertrand
    plain_message = "hello"
    encrypted_message = rsa_encrypt(plain_message, pq_dest, e_dest)
    print("Arielle : Sending to \"Bertrand\" : {}".format(encrypted_message))
    socket.send(encrypted_message.encode())

    # Arielle reçoit le message de Bertrand
    encrypted_message = socket.recv(1024).decode()
    plain_message = rsa_decrypt(encrypted_message, pq, d)
    print("Arielle : Received from \"Bertrand\" : {}".format(plain_message))

    # Déconnexion du client
    socket.close()
    print("Arielle : Disconnected")


port = 2048
serveur = threading.Thread(target=laurent, args=([port]))
serveur.start()
bertrand(port)
arielle(port)

# Partie 4


Laurent : initialized on port 2048Bertrand 55506: connected

Laurent : Listening
Arielle 55508: connected
Laurent : New connection from 127.0.0.1 at 55506
Laurent : Received from 55506 : (pq=2, e=2)
Laurent : New connection from 127.0.0.1 at 55508
Bertrand : Received from "Arielle" : (pq=1, e=1)
Laurent : Received from 55508 : (pq=3, e=3)
Arielle : Received from "Bertrand" : (pq=1, e=1)
Arielle : Sending to "Bertrand" : hello
Laurent : Intercepted hello from 55508
Laurent : Forwarding to : 55506
Arielle : Received from "Bertrand" : hello
Bertrand : Sending to "Arielle" : Hi
Laurent : Intercepted Hi from 55506
Laurent : Forwarding to : 55508
Bertrand : Disconnected
Arielle : Received from "Bertrand" : Hi
Arielle : Disconnected
Laurent : Disconnected


# 4. Malléabilité

*Laurent* n’a pas eu le temps de mettre en place sa stratégie d’attaque par homme du milieu, mais il peut cependant intercepter les messages transmis (il ne peut juste pas les déchiffrer). 

Il a accès aux clés publiques *RSA* d’*Arielle* et *Bertrand*, ainsi qu’aux messages chiffrés, qu’il peut intercepter, modifier et retransmettre.

Implémenter *Laurent* pour que, lorsqu’il intercepte un message chiffré ccorrespondant au message `m`(qu’il ne peut retrouver), il transmette à l’autre participant un message `m'` correspondant au message `2m`.

In [ ]:
def arielle():
    # connexion à Laurent
    
    # génération des clés + échange des clés publiques
    
    # envoi de messages chiffrés (+ affichage)
    
def bertrand():
    # connexion à Laurent
    
    # génération des clés + échange des clés publiques
    
    # déchiffrement des messages reçus (+ affichage)
    
def laurent():
    # connexion à Arielle et Bertrand
    
    # transmission de la clé d'Arielle à Bertrand (et inversement)
   
    # modification et transmission des messages interceptés

In [ ]:
import threading
import socket as sckt
import time

def rsa_encrypt(plain_message, pq, e) :
  encrypted_message = plain_message

  return encrypted_message

def rsa_decrypt(encrypted_message, pq, d) :
  plain_message = encrypted_message

  return plain_message

def rsa_malleable(encrypted_message, t):
  malleabilised_message = encrypted_message * t
  return malleabilised_message


def laurent(port):
  socket = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
  socket.bind(("127.0.0.1", port))
  
  socket.listen(True)
  
  print("Laurent : initialized on port {}".format(port))
  print("Laurent : Listening")
  
  # première victime
  conn1, addr1 = socket.accept()
  print('Laurent : Got new connection {}:{}'.format(addr1[0], addr1[1]))
  # réception des clés publiques de la première victime
  pq1 = int(conn1.recv(1024).decode())
  e1 = int(conn1.recv(1024).decode())
  print("Laurent : Received from {} : (pq={}, e={})".format(addr1[1], pq1, e1))
  
  # deuxième victime
  conn2, addr2 = socket.accept()
  print('Laurent : Got new connection {}:{}'.format(addr2[0], addr2[1]))
  # réception des clés publiques de la deuxième victime
  pq2 = int(conn2.recv(1024).decode())
  e2 = int(conn2.recv(1024).decode())
  print("Laurent : Received from {} : (pq={}, e={})".format(addr2[1], pq2, e2))
  
  # envoi des clés publique de Laurent à la première victime
  conn1.send(str(pq2).encode())
  time.sleep(1)
  conn1.send(str(e2).encode())

  # envoi des clés publique de Laurent à la première victime
  conn2.send(str(pq1).encode())
  time.sleep(1)
  conn2.send(str(e1).encode())

  def communication(conn_src, addr_src, conn_dest, addr_dest, pq_dest, e_dest):    
    # Interception d'un message des victimes
    encrypted_message = conn_src.recv(1024).decode()
    print("Laurent : Intercepted \"{}\" from {}".format(encrypted_message, addr_src[1]))
    
    # Transmettre le message à la deuxième victime
    print("Laurent : Forwarding to : {}".format(addr_dest[1]))
    malleabilised_message = rsa_malleable(encrypted_message, 2)
    conn_dest.send(malleabilised_message.encode())
       
    
    
  communication1 = threading.Thread(target=communication, args=(conn1, addr1, conn2, addr2, pq2, e2))
  communication1.start()
  communication2 = threading.Thread(target=communication, args=(conn2, addr2, conn1, addr1, pq1, e1))
  communication2.start()
  communication1.join()
  communication2.join()
  conn1.close()
  conn2.close()
  socket.listen(False)
  socket.close()
  print("Laurent : Disconnected")

def bertrand(port):
  def bertrand_behavior(port):    
    socket = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
    try :
      socket.connect(("127.0.0.1", port))
      print("Bertrand {}: connected".format(socket.getsockname()[1]))
    except :
      print("Bertrand : Connection Error")

    # Bertrand construit ses clés
    p, q = 2,2
    pq =2
    e =2
    d =2

    # Communication des clés public au destinataire
    socket.send(str(pq).encode())
    time.sleep(1)
    socket.send(str(e).encode())

    # Réception des clés publiques du destinataire
    pq_dest = int(socket.recv(1024).decode())
    e_dest = int(socket.recv(1024).decode())
    print("Bertrand : Received from \"Arielle\" : (pq={}, e={})".format(pq_dest, e_dest))

    
    # Bertrand reçoit le message d'Arielle 
    encrypted_message = socket.recv(1024).decode()
    plain_message = rsa_decrypt(encrypted_message, pq, d)
    print("Arielle : Received from \"Bertrand\" : {}".format(plain_message))

    # Bertrand envoie une réponse à Arielle
    plain_message = "Hi"
    encrypted_message = rsa_encrypt(plain_message, pq_dest, e_dest)
    print("Bertrand : Sending to \"Arielle\" : {}".format(encrypted_message))
    socket.send(encrypted_message.encode()) 


    # Déconnexion du client
    socket.close()
    print("Bertrand : Disconnected")
    
  x = threading.Thread(target=bertrand_behavior, args=([port]))
  x.start()

  return x
  

def arielle(port):
  socket = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
  try :
    socket.connect(("127.0.0.1", port))
    print("Arielle {}: connected".format(socket.getsockname()[1]))
  except :
    print("Arielle : Connection Error")

  # Arielle construit ses clés
  p, q = 3,3
  pq =3
  e =3
  d =3

  # Communication des clés public au destinataire
  time.sleep(1)
  socket.send(str(pq).encode())
  time.sleep(1)
  socket.send(str(e).encode())

  # Réception des clés publiques du destinataire
  pq_dest = int(socket.recv(1024).decode())
  e_dest = int(socket.recv(1024).decode())
  print("Arielle : Received from \"Bertrand\" : (pq={}, e={})".format(pq_dest, e_dest))

  # Arielle envoie un message à Bertrand
  plain_message = "hello"
  encrypted_message = rsa_encrypt(plain_message, pq_dest, e_dest)
  print("Arielle : Sending to \"Bertrand\" : {}".format(encrypted_message))
  socket.send(encrypted_message.encode())

  # Arielle reçoit le message de Bertrand 
  encrypted_message = socket.recv(1024).decode()
  plain_message = rsa_decrypt(encrypted_message, pq, d)
  print("Arielle : Received from \"Bertrand\" : {}".format(plain_message))


  # Déconnexion du client
  socket.close()
  print("Arielle : Disconnected")
  
  
port = 4096
serveur = threading.Thread(target=laurent, args=([port]))
serveur.start()
bertrand(port)
arielle(port)


Laurent : initialized on port 4096
Laurent : Listening
Arielle 49826: connectedBertrand 49828: connected

Laurent : Got new connection 127.0.0.1:49828
Laurent : Received from 49828 : (pq=2, e=2)
Laurent : Got new connection 127.0.0.1:49826
Laurent : Received from 49826 : (pq=3, e=3)
Bertrand : Received from "Arielle" : (pq=3, e=3)
Arielle : Received from "Bertrand" : (pq=2, e=2)
Arielle : Sending to "Bertrand" : hello
Laurent : Intercepted "hello" from 49826
Laurent : Forwarding to : 49828
Arielle : Received from "Bertrand" : hellohello
Bertrand : Sending to "Arielle" : Hi
Laurent : Intercepted "Hi" from 49828
Laurent : Forwarding to : 49826
Arielle : Received from "Bertrand" : HiHi
Bertrand : DisconnectedArielle : Disconnected

Laurent : Disconnected
